# Exploritory Data Analysis

This file will take all the EDA performed by the team and made to run self contained on files from the GitHub repo.

### Contents
1. High Level Look At The Datasets
    1. Client Supplied Datasets For Temperature And Demand
    2. Solar PV Data
    3. Population Data
    4. Rain Data
2. Dataset Interactions
    1. A Closer Look At Demand Over Time
    2. Demand With Weather And Datetime Data
    3. Multivariate Analysis


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import dateutil.easter as ea
from statsmodels.tsa.seasonal import MSTL
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2 import robjects
CCP = importr("CCP")
base = importr("base")
quant_pys = importr("QuantPsyc")

pd.plotting.register_matplotlib_converters()
%matplotlib inline
sns.set_theme()

ImportError: cannot import name 'MSTL' from 'statsmodels.tsa.seasonal' (C:\Users\Aaron\anaconda3\lib\site-packages\statsmodels\tsa\seasonal.py)

## High Level Look At The Datasets
### Client Supplied Datasets for Temperature and Demand
Here we will load in the client supplied datasets for temperature and demand and have a look at the overall structure of the data.

In [2]:
# Load in client datasets
temp_df = pd.read_csv('../data/raw/temperature_nsw.csv', names=['datetime', 'location', 'temp'], header=0)
temp_df['datetime'] = pd.to_datetime(temp_df['datetime'])
demand_df = pd.read_csv('../data/raw/totaldemand_nsw.csv', names=['datetime', 'region', 'demand'], header=0)
demand_df['datetime'] = pd.to_datetime(demand_df['datetime'])

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/temperature_nsw.csv'

First, let's look at the temperature dataset

In [ ]:
print(f'Number of null cells: {temp_df["temp"].isnull().sum()}')
print(temp_df.head(-1))
print(temp_df.describe())

We can see there is likely some record or sensor issue causing -9999 values. Let's have a look at the boxplot to see if we need to remove any other values.

In [ ]:
temp_df.boxplot('temp')

Looks like it is just the -9999 values that will need to be removed.

In [ ]:
temp_df = temp_df[temp_df['temp'] > -9999]
# Recreate boxplot with problem values removed
ax = temp_df.boxplot('temp')
plt.title('Boxplot of Temperature')
plt.ylabel('Temperature (°C)')
plt.savefig('../images/exploratory/temperature-boxplot.png')

In [ ]:
ax = temp_df.hist('temp')
plt.title('Temperature Histogram')
plt.xlabel('Temperature (°C)')
plt.ylabel('Frequency')
plt.savefig('../images/exploratory/temperature-histogram.png')

Temperature records seem to follow a fairly normal trend.
Let's now have a look at the timings of when records were made to look for irregularities in the dataset

In [ ]:
plt.hist(x=temp_df['datetime'].dt.minute)
plt.title('Temperature Timing Histogram')
plt.xlabel('Minute of Hour')
plt.ylabel('Frequency')
plt.xticks(np.arange(0,60,step=5))
plt.savefig('../images/exploratory/temperature-timing-histogram.png')

We can see there may be some irregularities in the way the data was recoded but for the purposes of our analysis, this should not raise any issues.

Let's now have a similar look at the demand dataset.

In [ ]:
print(f'Number of null cells: {demand_df["demand"].isnull().sum()}')
print(demand_df.head(-1))
print(demand_df.describe())

In [ ]:
ax = demand_df.boxplot('demand')
plt.title('Boxplot of Demand')
plt.ylabel('Demand (MW)')
plt.savefig('../images/exploratory/demand-boxplot.png')

We can see that the base load in demand is fairly consistent around 4000kW, were as there is much more variation and larger peaks in the maximum demand.

In [ ]:
ax = demand_df.hist('demand')
plt.title('Demand Histogram')
plt.xlabel('Demand (MW)')
plt.ylabel('Frequency')
plt.savefig('../images/exploratory/demand-histogram.png')

The histogram shows a similar skew to the data as the boxplot.
Again, let's have a look at the timings of when data was collected.

In [ ]:
plt.hist(x=demand_df['datetime'].dt.minute, bins=12)
plt.xticks(np.arange(0,60,step=5))
plt.ylim([110250, 110340])
plt.title('Demand Timing Histogram')
plt.xlabel('Minute of Hour')
plt.ylabel('Frequency')
plt.savefig('../images/exploratory/demand-timing-histogram.png')

Again, we can see some irregularities in how data was recorded, with more records being made on the hour, though for the purposes of our analysis it should not matter.
Let's have a quick look at how demand has changed year-on-year.

In [ ]:
demand_df['year'] = demand_df.datetime.dt.year
ax = sns.boxplot(data=demand_df, x='year', y='demand')
plt.title('Demand by Year')
plt.xlabel('Year')
plt.ylabel('Demand (MW)')
for item in ax.get_xticklabels():
    item.set_rotation(90)
plt.savefig('../images/exploratory/demand-by-year.png')

It looks like there may be a downward trend in demand, though it is difficult to say.

### Solar PV Data
We are interested in how residential, or small-scale, solar panels affect demand in NSW. Let's bring in some data from the Clean Energy Regulator (CER) and have a look at it. The data is spread across numerous .xlsx files that have data for all states so it will need to be filtered and combined before we can look at it.

In [ ]:
# Supress a waring from openpyxl
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
# Load in data from first workbook as starting point for df
# named solar08_df as we will only end up taking the 2008 data from this df
solar08_df = pd.read_excel('../data/raw/Postcode data for small-scale installations 2009 - all data.xlsx',
                           sheet_name='SGU-Solar', header=2)
solar08_df.head(-1)

In [ ]:
# Create a function for filtering for NSW postcodes only i.e. 2XXX
def nsw_mask(df):
    mask = (df['Small Unit Installation Postcode'] >= 2000) & (df['Small Unit Installation Postcode'] < 3000)
    return mask

In [ ]:
# Filter out unnecessary locations
mask08 = nsw_mask(solar08_df)
nsw08_df = solar08_df.loc[mask08]

# Get the initial data from prior to 2008
INITIAL_SOLARUNITS = nsw08_df.sum()['Previous Years (2001-2007) - Installations Quantity']
INITIAL_SOLARPOWER = nsw08_df.sum()['Previous Years (2001-2007) - SGU Rated Output In kW']

# Get total number of units added in usable format
solar_units = nsw08_df.filter(regex='2008 - Installations Quantity', axis=1).sum()
solar_units.index = ['Jan 2008', 'Feb 2008', 'Mar 2008', 'Apr 2008', 'May 2008', 'Jun 2008',
                     'Jul 2008', 'Aug 2008', 'Sep 2008', 'Oct 2008', 'Nov 2008', 'Dec 2008']
solar_units.index = pd.to_datetime(solar_units.index)

# Do the same for power added
solar_output = nsw08_df.filter(regex='2008 - SGU Rated Output In kW', axis=1).sum()
solar_output.index = ['Jan 2008', 'Feb 2008', 'Mar 2008', 'Apr 2008', 'May 2008', 'Jun 2008',
                      'Jul 2008', 'Aug 2008', 'Sep 2008', 'Oct 2008', 'Nov 2008', 'Dec 2008']
solar_output.index = pd.to_datetime(solar_output.index)

In [ ]:
# Add data from other workbooks
months = ['Jan ', 'Feb ', 'Mar ', 'Apr ', 'May ', 'Jun ',
          'Jul ', 'Aug ', 'Sep ', 'Oct ', 'Nov ', 'Dec ']

# Loop through all the years we have solar data for and add the data to df
for year in range(2009, 2021):
    df = pd.read_excel('../data/raw/Postcode data for small-scale installations ' + str(year) + ' - all data.xlsx',
                       sheet_name=0, header=2)
    # filter by NSW data only
    loc_mask = nsw_mask(df)
    df = df.loc[loc_mask]
    # separate Quantity and Output
    df_units = df.filter(regex= str(year) + ' - Installations Quantity', axis=1).sum()
    df_output = df.filter(regex= str(year) + ' - SGU Rated Output In kW', axis=1).sum()
    # Reset index to datetime
    month_year = []
    for month in months:
        month_year.append(month + str(year))
    df_units.index = month_year
    df_units.index = pd.to_datetime(df_units.index)
    df_output.index = month_year
    df_output.index = pd.to_datetime(df_output.index)
    # Append to end of dataframe
    solar_units = pd.concat([solar_units, df_units])
    solar_output = pd.concat([solar_output, df_output])

# Build usable dataset for export
cum_solar_units = solar_units.cumsum() + INITIAL_SOLARUNITS
cum_solar_output = solar_output.cumsum() + INITIAL_SOLARPOWER

solar_df = pd.concat([solar_units, cum_solar_units, solar_output, cum_solar_output], axis=1)
solar_df.rename(columns={0:'solar units added', 1:'cumulative units added', 2:'solar output added', 3:'cumulative output added'}, inplace=True)
solar_df.index.names = ['datetime']

In [ ]:
solar_df.head()

In [ ]:
ax = solar_df['solar units added'].plot(ylabel='Number of Units Added', legend=True)
solar_df['solar output added'].plot(ax=ax, secondary_y=True, ylabel='Output Added (kW)', legend=True)
plt.title('Number of Units and Energy Output Added From Residential Solar Panels in NSW Each Month')
plt.xlabel('Year')
plt.savefig('../images/exploratory/units-output-plot.png')

We can see that the number of units being added in the 2010-2012 period is quite high, which may be due to government schemes, rebates, or even some change in they way the recording was being done in that time. As we will most likely be interested in the cumulative effects of solar, this will likely have little affect on our modelling.
It is also worth noting that the Output being added per unit is increasing, meaning the technology is getting more efficient.

Now let's look at the cumulative values that we are more interested in.

In [ ]:
ax = solar_df['cumulative units added'].plot(ylabel='Cumulative Units Added', legend=True)
solar_df['cumulative output added'].plot(ax=ax, secondary_y=True, ylabel='Cumulative Output Added (kW)', legend=True)
plt.title('Cumulative Units and Energy Output Added From Residential Solar Panels in NSW Each Month')
plt.xlabel('Year')
plt.savefig('../images/exploratory/cumulative-units-output-plot.png')

We can see that the large spike in units and output added in the 2010-2012 period is still noticeable, but less impactful in the cumulative output that we will likely be the most interested in.
It's worth noting that this graph shows a similar trend in output growing faster than the units being added, but both look like the are growing at an increasing rate since the large addition in the 2010-2012 period.

### Population Data
Population will likely play a role in the demand. Let's bring in population data from the Australian Bureau of Statistics (ABS). The ABS data goes from 2001-2021 and is yearly.

In [ ]:
# Import population data
pop_df = pd.read_csv('../data/raw/NSW_population.csv', usecols=['TIME_PERIOD: Time Period', 'OBS_VALUE'], header=0)
pop_df.rename(columns={'TIME_PERIOD: Time Period':'datetime', 'OBS_VALUE':'population'}, inplace=True)
pop_df['datetime'] = pd.to_datetime(pop_df['datetime'], format='%Y')
pop_df.set_index('datetime', inplace=True)
pop_df.head(21)

In [ ]:
pop_df['population'].plot(title='Population in NSW', xlabel='Year', ylabel='Population', ylim=[0, 9e6])
plt.savefig('../images/exploratory/NSW-population.png')

We can see that population grows at a relatively steady rate. It is worth noting that 2021 had negative growth compared to all previous years, which were closer to 1%. This may be due to a correction to estimates from the census data. It will need to be considered when modelling and future forecasting, but these are likely the best estimates available.

### Rain Data
As we are interested in the effects of residential solar on the grid demand, we would like to incorporate cloud coverage data. As we don't have that we will use daily precipitation data from weather stations in areas with a lot of solar output as a proxy to see if it has an effect on grid demand due to changing residential solar output.

First, we will take some data from the Australian PV Institute to find regions by postcode that have high solar output.

In [ ]:
postcode_pv_df = pd.read_csv('../data/raw/Postcode time series.csv')
postcode_df_sum = postcode_pv_df.groupby('Postcode').agg('sum', numeric_only=True)
ax = postcode_df_sum.sort_values('Capacity (kW)').plot(kind='barh', title='Capacity by Postcode Region', legend=False, xlabel='Capacity (kW)')
plt.savefig('../images/exploratory/output-by-postcode.png')

From the results above, the 21XX postcodes have the highest PV installation capacity. From this [map](https://pv-map.apvi.org.au/historical#9/-33.9115/150.5814), we can see that 21XX postcodes are the suburbs near the Sydney CBD.

Now, we find the [BoM weather stations](http://www.bom.gov.au/nsw/observations/sydneymap.shtml) that are located in the 21XX suburbs, which are:

*   PARRAMATTA NORTH (MASONS DRIVE), 066124
*   KIAMA (BOMBO HEADLAND), 068242
*   SYDNEY OLYMPIC PARK AWS (ARCHERY CENTRE), 066212
*   CANTERBURY RACECOURSE AWS, 066194
*   HORSLEY PARK EQUESTRIAN CENTRE AWS, 067119
*   PROSPECT RESERVOIR, 067019

The daily rain data is then extracted from those stations ([link](http://www.bom.gov.au/climate/data/)), and then we extract the data from 01/01/2010 onwards.

In [ ]:
weather_station_ids = ['066124', '068242', '066212', '066194', '067119', '067019']

# Get all data from weather stations
for n in range(len(weather_station_ids)):
    file_name = 'IDCJAC0009_' + weather_station_ids[n] + '_1800_Data.csv'

    # Create new dataframe
    if n == 0:
        bom_df = pd.read_csv('../data/raw/' + file_name)
        bom_df = bom_df.drop(columns=['Product code', 'Period over which rainfall was measured (days)', 'Quality'])
        bom_df = bom_df[bom_df['Year'] >= 2010]
        bom_df['Rainfall amount (millimetres)'] = bom_df['Rainfall amount (millimetres)'].fillna(0)

    # Add to existing dataframe
    else:
        df = pd.read_csv('../data/raw/' + file_name)
        df = df.drop(columns=['Product code', 'Period over which rainfall was measured (days)', 'Quality'])
        df = df[df['Year'] >= 2010]
        df['Rainfall amount (millimetres)'] = df['Rainfall amount (millimetres)'].fillna(0)
        bom_df = pd.concat([bom_df, df], ignore_index=True)

In [ ]:
# Create a single datetime column
bom_df['datetime'] = pd.to_datetime(bom_df['Year'].astype(str) + '-' + bom_df['Month'].astype(str) + '-' + bom_df['Day'].astype(str))
bom_df['datetime'] = pd.to_datetime(bom_df['datetime'])
# Get the total amount of rain for all weather stations for each day
rain_df = bom_df.groupby(["datetime"])["Rainfall amount (millimetres)"].sum().to_frame()
rain_df.head()

In [ ]:
ax = rain_df.boxplot('Rainfall amount (millimetres)')
plt.title('Boxplot of Rainfall')
plt.ylabel('Rainfall Amount (mm)')
plt.savefig('../images/exploratory/rain-boxplot.png')

In [ ]:
ax = rain_df.hist('Rainfall amount (millimetres)', bins=50)
plt.title('Rainfall Histogram')
plt.xlabel('Rainfall Amount (mm)')
plt.ylabel('Frequency')
plt.savefig('../images/exploratory/rain-histogram.png')

From the histogram and boxplot, we can see that this is a very skewed dataset. Most values are 0 or close to with some very large outliers. This data may not be very useful for our model.

## Looking At Dataset Interactions
We are now going to combine the proposed datasets and look at the interactions between some of the potential predictors in the model. We are interested in looking at how daily maximum and minimum demand changes over time, this will require some interpolation of data that are on different time scales, namely the solar_df and pop_df.
As these data change at a relatively steady rate over the timescales that we will be interpolating, we will assume that a simple linear interpolation will suffice.

We will also add in some predictors based on the date to see if date factors influence demand, such as weekends and public holidays.


In [ ]:
# Get daily min and max demand
demand_df = demand_df.resample('D', on='datetime')['demand'].agg(['min', 'max'])
demand_df.rename(columns={'min':'demand_min', 'max':'demand_max'}, inplace=True)

# Get daily min and max temperatures
temp_df = temp_df.resample('D', on='datetime')['temp'].agg(['min', 'max'])
temp_df.rename(columns={'min':'temp_min', 'max':'temp_max', 'mean':'temp_mean'}, inplace=True)

# Interpolate solar data but remove the amount of units added in a month
solar_df.drop(['solar units added', 'solar output added'], axis=1, inplace=True)
solar_df.rename(columns={'cumulative units added': 'cum_units', 'cumulative output added': 'cum_output'}, inplace=True)
solar_df = solar_df.resample('D', convention='end').interpolate(method='linear')

# Interpolate population data
pop_df = pop_df.resample('D', convention='start').interpolate(method='linear')

# Rename rainfall for simplicity
rain_df.rename(columns={'Rainfall amount (millimetres)': 'rain'}, inplace=True)

In [ ]:
# Merge all data into one dataframe
all_data = demand_df.merge(temp_df, on='datetime', how='outer')
all_data = all_data.merge(solar_df, on='datetime', how='outer')
all_data = all_data.merge(pop_df, on='datetime', how='outer')
all_data = all_data.merge(rain_df, on='datetime', how='outer')
all_data.sort_index(inplace=True)
all_data.head()

We have sections that have large chunks of data missing so they wil have to be filtered out.

In [ ]:
bounds_mask = (all_data.index >= '2010-01-01') & (all_data.index < '2020-12-01')
all_data = all_data[bounds_mask]
all_data.head(-1)

# There are still some dates with missing temperature
all_data = all_data.dropna()

Let's now add in our date time features.

In [ ]:
def create_date_predictors(dataframe):
    """Function to create datetime features from a datetime index"""
    df = dataframe.copy()
    df['dayofweek'] = df.index.dayofweek
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    return df

In [ ]:
def public_holiday_check(new_date_check):
    """Function for finding public holidays in NSW"""
    new_date = pd.Timestamp(new_date_check)
    # New Year and Australia Day
    if new_date.month == 1:
        if new_date.weekday() not in (5,6):
            if new_date.day in (1,26):
                return 1
        elif new_date.weekday() == 5:
            if new_date.day in (3,28):
                return 1
        else:
            if new_date.day in (2,27):
                return 1
    if new_date.month == 3 & ea.easter(new_date.year).month ==3:
        if new_date.day in range(ea.easter(new_date.year).day-2,ea.easter(new_date.year).day+1):
            return 1
    # Anzac day
    if new_date.month == 4:
        if new_date.weekday() not in (5,6):
            if new_date.day == 25:
                return 1
        elif new_date.weekday() == 6:
            if new_date.day == 26:
                return 1
        else:
            if new_date.day == 27:
                return 1
        if ea.easter(new_date.year).month == 4:
            if new_date.day in range(ea.easter(new_date.year).day-2,ea.easter(new_date.year).day+1):
                return 1
    # king/queen birthday
    if new_date.month == 6 & new_date.day > 7 & new_date.day < 15 & new_date.weekday() == 0:
        return 1
    # Labour day
    if new_date.month == 10 & new_date.day < 8 & new_date.weekday() == 0:
        return 1
    # Christmas and boxing day
    if new_date.month == 12:
        if new_date.weekday() not in (5,6):
            if new_date.day in (25,26):
                return 1
        elif new_date.weekday() == 5:
            if new_date.day in (27,28):
                return 1
        else:
            if new_date.day in (26,27):
                return 1
    return 0

In [ ]:
# Add in datetime predictors
all_data = create_date_predictors(all_data)
all_data['public_holiday'] = all_data.index.to_frame()['datetime'].apply(lambda x: public_holiday_check(x))
all_data.head()

OK, we now have a dataframe with all our proposed predictors. Let's have a closer look at some of the interactions.

### A Closer Look At Demand Over Time
We saw previously that there may be a downward trend in demand, let's have a closer look at the demand when you are just looking at the minimum and maximum demand for a day.

In [ ]:
ax = all_data['demand_min'].plot(figsize=(10,5), ms=1, lw=1, ylabel='Demand (MW)', title='Mininmum and Maximum Daily Demand for NSW')
all_data['demand_max'].plot(ax=ax)
plt.xlabel('Year')
plt.legend(['Min Demand', 'Max Demand'], loc='upper right')
plt.savefig('../images/exploratory/min-max-demand.png')

It is quite difficult in this plot to see the overall trend in the data.
We will now do a timeseries decomposition to remove the noise and look at the trend. We won't tune the decomposition too much as we just want to get an idea of the broad trend.

In [ ]:
# Use MSTL from the statsmodels package to do multiple decomposition
mstl_max = MSTL(all_data['demand_max'], periods=[7, 365])
res_max = mstl_max.fit()

In [ ]:
plt.rc("figure", figsize=(8, 6))
plt.rc("font", size=13)
fig = res_max.plot()
fig.savefig('../images/exploratory/max-decomp.png')

In [ ]:
# Now do the same with the min demand
mstl_min = MSTL(all_data['demand_min'], periods=[7, 365])
res_min = mstl_min.fit()

In [ ]:
fig = res_min.plot()
fig.savefig('../images/exploratory/min-decomp.png')

It does look like there is a downward trend in both minimum and maximum demand, especially in the 2011-2014 period.

### Demand With Weather And Datetime Data
Let's now see if there are relationships between weather, datetime, and demand.

In [ ]:
fig, ax = plt.subplots(1,2, sharey=True, figsize=(10,5))
sns.scatterplot(data=all_data, x='temp_max', y='demand_max', ax=ax[0])
ax[0].set_xlabel('Temperature (°C)')
ax[0].set_title('Maximum Temperature')
sns.scatterplot(data=all_data, x='temp_min', y='demand_max', ax=ax[1])
ax[1].set_xlabel('Temperature (°C)')
ax[1].set_title('Minimum Temperature')
ax[0].set_ylabel('Demand (MW)')
fig.suptitle('Maximum Daily Demand Vs Daily Temperature')
plt.savefig('../images/exploratory/max-demand-temp.png')

In [ ]:
fig, ax = plt.subplots(1,2, sharey=True, figsize=(10,5))
sns.scatterplot(data=all_data, x='temp_max', y='demand_min', ax=ax[0])
ax[0].set_xlabel('Temperature (°C)')
ax[0].set_title('Maximum Temperature')
sns.scatterplot(data=all_data, x='temp_min', y='demand_min', ax=ax[1])
ax[1].set_xlabel('Temperature (°C)')
ax[1].set_title('Minimum Temperature')
ax[0].set_ylabel('Demand (MW)')
fig.suptitle('Minimum Daily Demand Vs Daily Temperature')
plt.savefig('../images/exploratory/min-demand-temp.png')

The maximum demand plotted against maximum temperature seems to have the most structure in the data. Minimum daily temperature with both minimum and maximum demand appear to have similar amounts of structure to the data. Maximum temperature with minimum demand appears to have the least strong relationship.

Now let's do the same plots but with colour for days of week and public holidays to see if we can account for some of the outliers.

In [ ]:
fig, ax = plt.subplots(1,2, sharey=True, figsize=(10,5))
sns.scatterplot(data=all_data, x='temp_max', y='demand_max', hue='dayofweek', ax=ax[0])
ax[0].set_xlabel('Temperature (°C)')
ax[0].set_title('Maximum Temperature')
sns.scatterplot(data=all_data, x='temp_min', y='demand_max', hue='dayofweek', ax=ax[1], legend=False)
ax[1].set_xlabel('Temperature (°C)')
ax[1].set_title('Minimum Temperature')
ax[0].set_ylabel('Demand (MW)')
fig.suptitle('Maximum Daily Demand Vs Daily Temperature With Day Of Week')
plt.savefig('../images/exploratory/max-demand-temp-dayweek.png')

In [ ]:
fig, ax = plt.subplots(1,2, sharey=True, figsize=(10,5))
sns.scatterplot(data=all_data, x='temp_max', y='demand_min', hue='dayofweek', ax=ax[0])
ax[0].set_xlabel('Temperature (°C)')
ax[0].set_title('Maximum Temperature')
sns.scatterplot(data=all_data, x='temp_min', y='demand_min', hue='dayofweek', ax=ax[1])
ax[1].set_xlabel('Temperature (°C)')
ax[1].set_title('Minimum Temperature')
ax[0].set_ylabel('Demand (MW)')
fig.suptitle('Minimum Daily Demand Vs Daily Temperature With Day Of Week')
plt.savefig('../images/exploratory/min-demand-temp-dayweek.png')

It looks like the maximum daily demand is highly dependent on the day of the week, with Saturdays (5) and Sundays (6) tending to have lower demand than weekdays (0-4).
The minimum daily demand shows less dependency on the day of the week, but it still looks to be a factor.

In [ ]:
fig, ax = plt.subplots(1,2, sharey=True, figsize=(10,5))
sns.scatterplot(data=all_data, x='temp_max', y='demand_max', hue='public_holiday', alpha=0.6, ax=ax[0])
ax[0].set_xlabel('Temperature (°C)')
ax[0].set_title('Maximum Temperature')
sns.scatterplot(data=all_data, x='temp_min', y='demand_max', hue='public_holiday', alpha=0.6, ax=ax[1], legend=False)
ax[1].set_xlabel('Temperature (°C)')
ax[1].set_title('Minimum Temperature')
ax[0].set_ylabel('Demand (MW)')
fig.suptitle('Maximum Daily Demand Vs Daily Temperature With Public Holidays')
plt.savefig('../images/exploratory/max-demand-temp-pubhol.png')

In [ ]:
fig, ax = plt.subplots(1,2, sharey=True, figsize=(10,5))
sns.scatterplot(data=all_data, x='temp_max', y='demand_min', hue='public_holiday', alpha=0.6, ax=ax[0])
ax[0].set_xlabel('Temperature (°C)')
ax[0].set_title('Maximum Temperature')
sns.scatterplot(data=all_data, x='temp_min', y='demand_min', hue='public_holiday', alpha=0.6, ax=ax[1], legend=False)
ax[1].set_xlabel('Temperature (°C)')
ax[1].set_title('Minimum Temperature')
ax[0].set_ylabel('Demand (MW)')
fig.suptitle('Minimum Daily Demand Vs Daily Temperature With Public Holidays')
plt.savefig('../images/exploratory/min-demand-temp-pubhol.png')

It looks like Public Holidays have a similar effect as weekends.

Now let's see if the rain data has an effect. There are a lot of days that have no rain, the structure might be more visible if we filter out days that got less than 5mm of rain.

In [ ]:
fig, ax = plt.subplots(1,2, sharey=True, figsize=(10,5))
sns.scatterplot(data=all_data[all_data['rain'] > 5], x='temp_max', y='demand_max', hue='rain', ax=ax[0])
ax[0].set_xlabel('Temperature (°C)')
ax[0].set_title('Maximum Temperature')
sns.scatterplot(data=all_data[all_data['rain'] > 5], x='temp_min', y='demand_max', hue='rain', ax=ax[1], legend=False)
ax[1].set_xlabel('Temperature (°C)')
ax[1].set_title('Minimum Temperature')
ax[0].set_ylabel('Demand (MW)')
fig.suptitle('Maximum Daily Demand Vs Daily Temperature With Rain')
plt.savefig('../images/exploratory/max-demand-temp-rain.png')

In [ ]:
fig, ax = plt.subplots(1,2, sharey=True, figsize=(10,5))
sns.scatterplot(data=all_data[all_data['rain'] > 5], x='temp_max', y='demand_min', hue='rain', ax=ax[0])
ax[0].set_xlabel('Temperature (°C)')
ax[0].set_title('Maximum Temperature')
sns.scatterplot(data=all_data[all_data['rain'] > 5], x='temp_min', y='demand_min', hue='rain', ax=ax[1], legend=False)
ax[1].set_xlabel('Temperature (°C)')
ax[1].set_title('Minimum Temperature')
ax[0].set_ylabel('Demand (MW)')
fig.suptitle('Minimum Daily Demand Vs Daily Temperature With Rain')
plt.savefig('../images/exploratory/min-demand-temp-rain.png')

In [ ]:
fig, ax = plt.subplots(1,2, sharey=False, figsize=(10,5))
sns.scatterplot(data=all_data, x='rain', y='demand_max', ax=ax[0])
sns.scatterplot(data=all_data, x='rain', y='demand_min', ax=ax[1])
ax[0].set_xlabel('Rainfall (mm)')
ax[0].set_title('Maximum Demand')
ax[0].set_ylabel('Demand (MW)')
ax[1].set_xlabel('Rainfall (mm)')
ax[1].set_title('Minimum Demand')
ax[1].set_ylabel('Demand (MW)')
fig.suptitle('Demand and Rainfall')
plt.savefig('../images/exploratory/demand-rain.png')

Looks like rain does not influence demand very much and may not be a good proxy for cloud cover if cloud cover does influence grid load. It may be that data from more weather stations would be needed to approximate cloud coverage.

### Multivariate Analysis
Now, let's have a look at how some of the features are linked by looking at the Canonical Correlation.

In [ ]:
# Firstly scale the data for CCA
scaler = StandardScaler()
norm_data = scaler.fit_transform(all_data)
# Split into response and predictor data
y_data = norm_data[:,:2]
x_data = norm_data[:,2:]

In [ ]:
multivariate_normality(norm_data,alpha = 0.05)

In [ ]:
# Canonical Correlation Analysis
ca = CCA()
X_c, Y_c = ca.fit_transform(x_data, y_data)

In [ ]:
# Correlation between the first canonical pair
np.corrcoef(X_c[:,0], Y_c[:,0])[0,1]

In [ ]:
# Prepare dataframe for plotting heat map of correlations
ccX_df = pd.concat([all_data, pd.DataFrame({'CCX':X_c[:, 0], 'CCY':Y_c[:, 0]}, index=all_data.index)], axis=1)

In [ ]:
# Correlation between all fields , this will help analysis the relationship between the Transformed predictors and the responses
corr_X_df = ccX_df.corr(method='pearson')
plt.rc('font', size=10)
plt.figure(figsize=(10,8))
X_df_lt = corr_X_df.where(np.tril(np.ones(corr_X_df.shape)).astype(bool))
sns.heatmap(X_df_lt, cmap='coolwarm', annot=True, fmt='.1g')
plt.tight_layout()
plt.title('Heat Map with Predictor Canonical Correlations')
plt.savefig('../images/exploratory/heatmap-cca-preds.png')

We can see that there appears to be strong relationships between the temperature variables and the first canonical correlation, where as the date variables seem to be loading on the second canonical correlation; day of the week seems to have a strong influence, backing up what we saw earlier in the scatter plots. It seems that solar output and population may have some contributions to models, but it would be minor compared to temperature and datetime predictors.
Rain data still looks like it has little impact and can likely be dropped, as can day of the month.

It is also worth noting that there appears to be a high amount of correlation with some of our predictors, as is often the case with timeseries data. To mitigate this, cumulative units will be dropped. We may have to be mindful of how entangled population, year, and cumulative output are.

In [ ]:
A = CCP.p_perm(x_data,y_data,type = "Wilks")

In [ ]:
# Remove data that will not be needed for model building
all_data.drop(['cum_units', 'rain', 'dayofmonth'], axis=1, inplace=True)
all_data

In [ ]:
# Output our clean data for modelling
all_data.to_csv('../data/processed/all_data.csv')